### Part 3: Trajectory Evaluation and g2o

_Refer to the example notebooks for installation instructions_

### Evo

So you've implemented 2D SLAM, great! Now, what? We need a measure of how good the trajectory is. The error/loss used earlier doesn't tell us much about how the trajectory differs from the ground truth. Here, we try to do just this - compute error metrics. Rather than computing these from scratch, we will just Evo - https://github.com/MichaelGrupp/evo/.

Look at the absolute pose error (APE) and relative pose error (RPE). What do they capture and how are they calculated (descriptive answer)? How do these metrics differ in methodology? Can we determine if the error is more along the x/y axis?

Answer the above questions and report errors for the obtained trajectory.

In [68]:
# Answer the above questions. Also include plots/images.

Try to play around with this tool and add any other plots that you think might be relevant/interesting.

### g2o

Install g2o as mentioned in `examples/g2o.ipynb` and optimise `edges.txt`, the file you used earlier. Also use `g2o_viewer` and optimize `intel` (a trajectory in the Intel research lab) and `sphere`. They should look something like:


<table><tr>
<td> <img src="../misc/intel.jpg" alt="Drawing" style="width: 250px;"/> </td>
<td> <img src="../misc/sphere.jpg" alt="Drawing" style="width: 250px;"/> </td>
</tr></table>

## Done intel.g2o in the Part-2

Write briefly about your observations and try out few options in the GUI. What do they do, how do they perform?

In [2]:
import sys
sys.path.append("..")
import open3d as o3d
import numpy as np
from matplotlib import pyplot as plt
from misc import g2o_to_kitti as g2o

In [2]:
file = open("../data/sphere.g2o")
lines = file.readlines()

In [36]:
def readEdge(fileName):
    f = open(fileName, 'r')
    A = f.readlines()
    f.close()

    e1 = []
    e2 = []
    x_arr = []
    y_arr = []
    z_arr = []
    q_w_arr = []
    q_x_arr = []
    q_y_arr = []
    q_z_arr = []

    info_mat = []

    for line in A:
        if "EDGE_SE3:QUAT" in line:
            splits = line.split()
            e1.append(int(splits[1]))
            e2.append(int(splits[2]))
            x = splits[3]
            y = splits[4]
            z = splits[5]
            q_x = splits[6]
            q_y = splits[7]
            q_z = splits[8]
            q_w = splits[9]
            # forming a symmetric matrices from float(splits[10]) onwards

            mat = [
                [float(splits[10]), float(splits[11]), float(splits[12]), float(splits[13]), float(splits[14]), float(splits[15])],
                [float(splits[11]), float(splits[16]), float(splits[17]), float(splits[18]), float(splits[19]), float(splits[20])],
                [float(splits[12]), float(splits[17]), float(splits[21]), float(splits[22]), float(splits[23]), float(splits[24])],
                [float(splits[13]), float(splits[18]), float(splits[22]), float(splits[25]), float(splits[26]), float(splits[27])],
                [float(splits[14]), float(splits[19]), float(splits[23]), float(splits[26]), float(splits[28]), float(splits[29])],
                [float(splits[15]), float(splits[20]), float(splits[24]), float(splits[27]), float(splits[29]), float(splits[30])]
            ]
            
            info_mat.append(mat)

            
            x_arr.append(float(x) / 1000)
            y_arr.append(float(y) / 1000)
            z_arr.append(float(z) / 1000)
            q_w_arr.append(float(q_w))
            q_x_arr.append(float(q_x))
            q_y_arr.append(float(q_y))
            q_z_arr.append(float(q_z))

    return np.array([np.array(e1), np.array(e2), np.array(x_arr), np.array(y_arr), np.array(z_arr), np.array(q_w_arr), np.array(q_x_arr), np.array(q_y_arr), np.array(q_z_arr)]),  np.array(info_mat)

In [38]:
inp, info_mats = readEdge('../data/sphere.g2o')
i1 = inp[0].astype(np.int32)
i2 = inp[1].astype(np.int32)
x = inp[2]
y = inp[3]
z = inp[4]
q_w = inp[5]
q_x = inp[6]
q_y = inp[7]
q_z = inp[8]

In [39]:
def quaternion_to_mat(q):
    w, x, y, z = q

    R = np.array([
        [1 - 2*y**2 - 2*z**2, 2*x*y - 2*z*w, 2*x*z + 2*y*w],
        [2*x*y + 2*z*w, 1 - 2*x**2 - 2*z**2, 2*y*z - 2*x*w],
        [2*x*z - 2*y*w, 2*y*z + 2*x*w, 1 - 2*x**2 - 2*y**2]
    ])

    return R

import jax.numpy as jnp

def quaternion_to_mat_jac(q):
    w, x, y, z = q

    R = jnp.array([
        [1 - 2*y**2 - 2*z**2, 2*x*y - 2*z*w, 2*x*z + 2*y*w],
        [2*x*y + 2*z*w, 1 - 2*x**2 - 2*z**2, 2*y*z - 2*x*w],
        [2*x*z - 2*y*w, 2*y*z + 2*x*w, 1 - 2*x**2 - 2*y**2]
    ])

    return R

def quaternion_product(q, p):
    a0 = q[0]*p[0] - q[1]*p[1] - q[2]*p[2] - q[3]*p[3]
    a1 = q[0]*p[1] + q[1]*p[0] + q[2]*p[3] - q[3]*p[2]
    a2 = q[0]*p[2] + q[2]*p[0] - q[1]*p[3] + q[3]*p[1]
    a3 = q[0]*p[3] + q[3]*p[0] + q[1]*p[2] - q[2]*p[1]
    return np.array([a0, a1, a2, a3])

def quaternion_product_jac(q, p):
    a0 = q[0]*p[0] - q[1]*p[1] - q[2]*p[2] - q[3]*p[3]
    a1 = q[0]*p[1] + q[1]*p[0] + q[2]*p[3] - q[3]*p[2]
    a2 = q[0]*p[2] + q[2]*p[0] - q[1]*p[3] + q[3]*p[1]
    a3 = q[0]*p[3] + q[3]*p[0] + q[1]*p[2] - q[2]*p[1]
    return jnp.array([a0, a1, a2, a3])

In [40]:
def visualize(i1, i2, x, y, z, q_w, q_x, q_y, q_z, vis = True):
    if(vis is True):
        vis = o3d.visualization.Visualizer()
        vis.create_window()
    
    p1 = np.array([0, 0, 0, 1, 0, 0, 0]) # x, y, z, qx, qy, qz
    num = i1.shape[0]
    num = len(np.unique(np.concatenate([i1, i2])))
    # num = 2
    p = np.zeros((num + 1, 7))
    p[0] = p1
    for i in range(num):
        mat = quaternion_to_mat((p[i, 3], p[i, 4], p[i, 5], p[i, 6]))
        new = mat @ np.array([x[i], y[i], z[i]])
        final = new + p[i][:3]
        p[i+1, :3] = final
        # p[i + 1, 3:] = quaternion_product(p[i, 3:], np.array([q_w[i], q_x[i], q_y[i], q_z[i]]))
        p[i + 1, 3:] = quaternion_product(p[i, 3:], np.array([q_w[i], q_x[i], q_y[i], q_z[i]]))
        # print("after", p[i + 1, 3:])


    if(vis is True):
        coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=5000, origin=[0, 0, 0])
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(p[:, :3])
        vis.add_geometry(pcd)
        # vis.add_geometry(coord)


        
        vis.run()
        vis.destroy_window()
    
    return p[1:, :]

visualize(i1, i2, x, y, z, q_w, q_x, q_y, q_z)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


array([[-2.96098000e-03,  2.36532000e-02,  4.51376000e-03,
         8.06812000e-01,  5.78115000e-01,  4.89471000e-02,
         1.11546000e-01],
       [-1.16798811e-02,  2.15643889e-02,  4.11536961e-02,
         3.04440849e-01,  9.21588986e-01,  3.63457239e-02,
         2.38052012e-01],
       [-1.82836708e-02, -2.84168890e-02,  4.93136233e-02,
        -3.04665373e-01,  9.01589188e-01, -5.83478478e-02,
         3.01517406e-01],
       [-1.70635287e-02, -5.58281000e-02, -8.29151923e-03,
        -7.89921070e-01,  5.40122353e-01, -1.96068539e-01,
         2.14129505e-01],
       [-1.86775444e-02,  8.59112900e-04, -5.83246174e-02,
        -9.53966449e-01,  7.17689767e-03, -2.99719792e-01,
        -8.13891178e-03],
       [-3.56440053e-02,  7.41867389e-02, -1.61073552e-02,
        -7.53988409e-01, -5.02659131e-01, -2.98876674e-01,
        -2.99179785e-01],
       [-5.86093050e-02,  5.08793964e-02,  7.37734549e-02,
        -2.70047321e-01, -7.96304868e-01, -1.50496945e-01,
        -5.1992797

In [41]:
def getJacobian(i1, i2, dX, dY, dZ, dq1, dq2, dq3, final):
    '''
    gives the Jacobian
    '''
    nodes = np.unique(np.concatenate([i1, i2]))

    num_edges = i1.shape[0]
    jacobian = np.zeros((6 * (num_edges + 1), 6 * len(nodes)))
    dq0 = np.sqrt(1 - dq1**2 - dq2**2 - dq3**2)
    i1 = i1.astype(np.int64)
    i2 = i2.astype(np.int64)
    for i in range(num_edges):
        n1, n2 = i1[i], i2[i]
        q1, q2, q3 = final[n1, 3:]
        if(q1 **2 + q2 ** 2 + q3 **2 >= 1):
            sqrt = np.sqrt(q1 **2 + q2 ** 2 + q3 **2 + 1e-5)
            q1 = q1 / sqrt
            q2 = q2 / sqrt
            q3 = q3 / sqrt
        q0 = np.sqrt(1 - (q1**2) - (q2**2) - (q3**2) + 1e-5)
        # print(1 - (q1**2) - (q2**2) - (q3**2))
        mat1 = np.eye(6)
        mat2 = -np.eye(6)
        dtrans = np.array([dX[i], dY[i], dZ[i]])
        transJacRotX = np.array([[0, 2*q2 + 2*q3*q1/q0, 2*q3 - 2*q2*q1/q0],
                                [-4*q2, 2*q1 + 2*q3*q2/q0, 2*q0 - 2*q2*q2/q0],
                                [-4*q3, -2*q0 + 2*q3*q3/q0, 2*q1 - 2*q2*q3/q0]])
        
        transJacRotY = np.array([[2*q2 - 2*q3*q1/q0, -4*q1, 2*q1*q1/q0 - 2*q0],
                                [2*q1 - 2*q3*q2/q0, 0, 2*q3 + 2*q1*q2/q0],
                                [2*q0 - 2*q3*q3/q0, -4*q3, 2*q2 + 2*q1*q3/q0]])
        
        transJacRotZ = np.array([[2*q3 + 2*q1*q2/q0, 2*q0 -2*q1*q1/q0, -4*q1],
                                [-2*q0 + 2*q2*q2/q0, 2*q3 - 2*q1*q2/q0, -4*q2],
                                [2*q1 + 2*q2*q3/q0, 2*q2 - 2*q1*q3/q0, 0]])
        
        mat1[0, 3:] = transJacRotX @ dtrans
        mat1[1, 3:] = transJacRotY @ dtrans
        mat1[2, 3:] = transJacRotZ @ dtrans

        dQX, dQY, dQZ = dq1[i], dq2[i], dq3[i]
        if(1 - (dQX**2) - (dQY**2) - (dQZ**2) < 0):
            dQX = dQX / np.sqrt((dQX**2) + (dQY**2) + (dQZ**2))
            dQY = dQY / np.sqrt((dQX**2) + (dQY**2) + (dQZ**2))
            dQZ = dQZ / np.sqrt((dQX**2) + (dQY**2) + (dQZ**2))
        dQW = np.sqrt(1 - (dQX**2) - (dQY**2) - (dQZ**2) + 1e-5)
        # we are now computing gradients, with
        # a1 = qk[0]*dq[1] + qk[1]*dq[0] + qk[2]*dq[3] - qk[3]*dq[2], a2 = qk[0]*dq[2] + qk[2]*dq[0] - qk[1]*dq[3] + qk[3]*dq[1], a3 = qk[0]*dq[3] + qk[3]*dq[0] + qk[1]*dq[2] - qk[2]*dq[1]
        mat1[3:, 3:] = np.array([[dQW - dQX*q1/q0, dQZ - dQX*q2/q0, -dQY - dQX*q3/q0,],
                                 
                                 [-dQZ - dQY*q1/q0, dQW - dQY*q2/q0, dQX - dQY*q3/q0],

                                 [dQY - dQZ*q1/q0, -dQX - dQZ*q2/q0, dQW - dQZ*q3/q0]])

        
        jacobian[i * 6 : (i + 1) * 6, n1 * 6 : (n1 + 1) * 6] = mat1
        jacobian[i * 6 : (i + 1) * 6, n2 * 6 : (n2 + 1) * 6] = mat2
    
    jacobian[-3 : , 0 : 3] = np.eye(3)
    return jacobian


def getInformationMatrix(I, anchorMat):
    '''
    gives the information matrix
    '''
    num_edges = I.shape[0]
    info_mat = np.zeros((6 * (num_edges + 1), 6 * (num_edges + 1)))
    for i in range(num_edges):
        info_mat[i * 6 : (i + 1) * 6, i * 6 : (i + 1) * 6] = I[i]
    
    info_mat[-6:, -6:] = anchorMat
    
    return info_mat


In [54]:
from scipy.linalg import cho_factor, cho_solve
import jax.numpy as jnp
from jax import jacfwd
import numpy as np

def doSLAMGauss3d(i1, i2, dX, dY, dZ, dq0, dq1, dq2, dq3, I):

    iterations = 2700
    
    lr = 0.001
    loss_arr = []
    # final = graph.copy()
    p = visualize(i1, i2, x, y, z, dq0, dq1, dq2, dq3, False)
    nodes = np.unique(np.concatenate([i1, i2]))
    final_ = p[:, :3]
    _final = p[:, 4:]
    final = np.column_stack([final_, _final])
    # final = np.random.randn(len(nodes), 6)

    def g(x):
        f = jnp.zeros(6 * (i1.shape[0] + 1))
        x = x.reshape(-1, 6)
        for i in range(i1.shape[0]): # iterating thorugh all constraints
            n1 = i1[i]
            n2 = i2[i]
            mat = quaternion_to_mat_jac((jnp.sqrt(1 - (x[n1, 3] ** 2) - (x[n1, 4] ** 2) - (x[n1, 5] ** 2)), x[n1, 3], x[n1, 4], x[n1, 5]))
            precalc = mat @ jnp.array([dX[i], dY[i], dZ[i]])
            newX = x[n1, 0] - x[n2, 0] + precalc[0]
            newY = x[n1, 1] - x[n2, 1] + precalc[1]
            newZ = x[n1, 2] - x[n2, 2] + precalc[2]
            qw, qx, qy, qz = quaternion_product_jac((jnp.sqrt(1 - (x[n1, 3] ** 2) - (x[n1, 4] ** 2) - (x[n1, 5] ** 2)), x[n1, 3], x[n1, 4], x[n1, 5]), (dq0[i], dq1[i], dq2[i], dq3[i]))
            
            newQx = qx - x[n2, 3] + x[n1, 4] ** 2
            newQy = qy - x[n2, 4] + x[n1, 4] ** 2
            newQz = qz - x[n2, 5] + x[n1, 5] ** 2
            
            f = f.at[6 * i : 6 * (i+1)].set(jnp.array([newX, newY, newZ, newQx, newQy, newQz]))
            # print(newT)
            # newT = np.sin(newT / 2)
            # newT = newT * np.sin(4 * np.pi ** 2 / newT)
            # f = f.at[6 * i : 6 * (i+1)].set(jnp.array([newX, newY, newZ, newQx, newQy, newQz]))
        
        f = f.at[-6:].set(x[0])
        return f

    jac = jacfwd(g)(jnp.array(final).reshape(-1))
    jacob = getJacobian(i1, i2, dX, dY, dZ, dq1, dq2, dq3, final)
    print("hi", np.linalg.norm(jac - jacob))
    
    for iteration in range(iterations):
        
        jacob = getJacobian(i1, i2, dX, dY, dZ, dq1, dq2, dq3, final)
        anchor_mat = np.eye(6) * 40000

        info = getInformationMatrix(I, anchor_mat)
        
        lhs = jacob.T @ info @ jacob
        precalc = -1 * jacob.T @ info.T
        
        f = np.empty(6 * (i1.shape[0] + 1))
        for i in range(i1.shape[0]):
            n1 = i1[i]
            n2 = i2[i]
            mat = quaternion_to_mat((dq0[i], dq1[i], dq2[i], dq3[i]))
            newX = final[n1, 0] - final[n2, 0] + ((mat @ np.array([dX[i], dY[i], dZ[i]]))[0])
            newY = final[n1, 1] - final[n2, 1] + ((mat @ np.array([dX[i], dY[i], dZ[i]]))[1])
            newZ = final[n1, 2] - final[n2, 2] + ((mat @ np.array([dX[i], dY[i], dZ[i]]))[2])
            qx = final[n1, 3]
            qy = final[n1, 4]
            qz = final[n1, 5]
            if(qx **2 + qy ** 2 + qz **2 >= 1):
                sqrt = np.sqrt(qx **2 + qy ** 2 + qz **2)
                qx = qx / sqrt
                qy = qy / sqrt
                qz = qz / sqrt
            qw, qx, qy, qz = quaternion_product((np.sqrt(1 - (qx ** 2) - (qy ** 2) - (qz ** 2) + 1e-5), qx, qy, qz), (dq0[i], dq1[i], dq2[i], dq3[i]))
            newQx = qx - final[n2, 3]
            newQy = qy - final[n2, 4] 
            newQz = qz - final[n2, 5] 
            f[6 * i : 6 * (i+1)] = np.array([newX, newY, newZ, newQx, newQy, newQz])
            
        f[-6:] = final[0]
        
        rhs = precalc @ f
        # c, low = cho_factor(lhs)
        # dx = cho_solve((c, low), rhs)
        dx = np.linalg.solve(lhs, rhs)
        # print(dx.reshape(-1, 6)[:, :3])
        final += lr * dx.reshape(-1, 6)

        loss = f.T @ info @ f
        loss_arr.append(loss)
        print(iteration, loss)
            
    # print(loss_arr[-1])
    # plt.plot(loss_arr)
    # plt.show()
    # fig, ax = plt.subplots(1, 2, figsize=(12, 4))
    # plt.suptitle("Final Graph Construction")
    # ax[0].scatter(graph[:,0], graph[:,1])
    # ax[1].scatter(final[:,0], final[:,1])
    # draw(final[:, 0], final[:, 1], final[:, 2])
    # for k in range(final.shape[0]):
    #     plt.scatter(final[:k, 0], final[:k, 1])
    #     plt.show()
        
    # plt.show()
    return final
        
bccc, I = readEdge("../data/sphere.g2o")
# bccc, I = readEdge("../data/edges3.txt")

i1, i2, X, Y, Z, dw, dx, dy, dz = bccc
X = X / 1000
Y = Y / 1000
Z = Z / 1000
i1 = i1.astype(np.int64)
i2 = i2.astype(np.int64)
# getJacobian(i1, i2)
optimizedGraph = doSLAMGauss3d(i1, i2, X, Y, Z, dw, dx, dy, dz, I)

hi 14.837511
0 6420677.111655898
1 6415387.038375788
2 6410079.085770105
3 6404753.108604798
4 6399408.967014257
5 6394046.526640961
6 6388665.658729123
7 6383266.240182377
8 6377848.153627202
9 6372411.287639204
10 6366955.537781376
11 6361480.479572866
12 6355986.672712475
13 6350474.269418269
14 6344943.22564156
15 6339393.504824786
16 6333825.0778172985
17 6328237.922774126
18 6322632.025040151
19 6317007.377022003
20 6311363.978049789
21 6305701.834230617
22 6300020.958295648
23 6294321.369442169
24 6288603.093171978
25 6282866.161127143
26 6277110.61092401
27 6271336.485986171
28 6265543.83537687
29 6259732.713631264
30 6253903.18058884
31 6248055.301226113
32 6242189.145489741
33 6236304.7881301455
34 6230402.308535639
35 6224481.790567106
36 6218543.322393246
37 6212586.996326451
38 6206612.908659315
39 6200621.159501929
40 6194611.8526200205
41 6188585.095274098
42 6182540.998059805
43 6176479.674749611
44 6170401.242136129
45 6164305.81987728
46 6158193.53034353
47 6152064.49

In [43]:
print(optimizedGraph[0])
print(optimizedGraph[1])
print(optimizedGraph[2])


[-0.05966417  0.02086603 -0.00665218  0.5569323   0.00090569  0.11641211]
[-0.06754945  0.01897814  0.02648274  0.90848362  0.00163423  0.25911118]
[-0.07352226 -0.02622052  0.03386264  0.86889358 -0.08146329  0.32190994]


In [55]:
vis = o3d.visualization.Visualizer()
vis.create_window()

coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=5000, origin=[0, 0, 0])
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(optimizedGraph[:, :3])
vis.add_geometry(pcd)
# vis.add_geometry(coord)


vis.run()
vis.destroy_window()

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
